In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb  5 11:10:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

##  data set


### Setup and Load dataset

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown https://drive.google.com/uc?id=1IaTQjGzKzbbu6uoISwTx_J4XuJLTwr52

Downloading...
From: https://drive.google.com/uc?id=1IaTQjGzKzbbu6uoISwTx_J4XuJLTwr52
To: /content/Mobile_Frappe.zip
100% 3.37M/3.37M [00:00<00:00, 264MB/s]


In [ ]:
!unzip "Mobile_Frappe.zip"  -d  "/content"

Archive:  Mobile_Frappe.zip
replace /content/Mobile_Frappe/frappe/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/Mobile_Frappe/frappe/.DS_Store  
  inflating: /content/Mobile_Frappe/frappe/.ipynb_checkpoints/stats-checkpoint.ipynb  
  inflating: /content/Mobile_Frappe/frappe/frappe.csv  
  inflating: /content/Mobile_Frappe/frappe/meta.csv  
  inflating: /content/Mobile_Frappe/frappe/stats.ipynb  
  inflating: /content/Mobile_Frappe/README.txt  


In [ ]:
import pandas
rating_df = pandas.read_csv('Mobile_Frappe/frappe/frappe.csv', sep="\t")
rating_df.head()

,user,item,cnt,daytime,weekday,isweekend,homework,cost,weather,country,city
0,0,0,1,morning,sunday,weekend,unknown,free,sunny,United States,0
1,1,1,7,afternoon,saturday,weekend,unknown,free,cloudy,Spain,0
2,2,2,6,evening,monday,workday,unknown,free,cloudy,Spain,369
3,3,3,1,sunset,thursday,workday,unknown,free,unknown,United States,1028
4,4,4,428,night,thursday,workday,home,free,sunny,Switzerland,147


In [ ]:
item_df = pandas.read_csv('Mobile_Frappe/frappe/meta.csv', sep="\t")
item_df.head(2)

,item,package,category,downloads,developer,icon,language,description,name,price,rating,short desc
0,0,com.anydo,Productivity,"1,000,000 - 5,000,000",Any.DO,http://d2lh3rxs7crswz.cloudfront.net/com.anydo...,en,"Meet Any.DO, the best way to-do To-do's on And...",Any.DO To-do & Tasks List,Free,4.5,Any.DO helps you remember everything you have ...
1,1,com.yahoo.mobile.client.android.yahoo,News & Magazines,"1,000,000 - 5,000,000",Yahoo! Inc.,http://d2lh3rxs7crswz.cloudfront.net/com.yahoo...,en,"With Yahoo! for Android, you’ll stay connected...",Yahoo!,Free,4.2,"With Yahoo! for Android, you’ll stay connected..."


In [ ]:
item_df=item_df[['item']]

In [ ]:
rating_df = pd.merge(rating_df, item_df,  how='left', left_on=['item'], right_on = ['item'])


In [ ]:
rating_df=rating_df.reset_index(drop=True)

In [ ]:

rating_df.rename(columns = {'cnt':'label'}, inplace = True)

In [ ]:
rating_df.head(1)

,user,item,label,daytime,weekday,isweekend,homework,cost,weather,country,city,category,language
0,0,0,1,morning,sunday,weekend,unknown,free,sunny,United States,0,Productivity,en


## LibRecommender

In [ ]:
!pip install numpy==1.22

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 74.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


In [ ]:
!pip install LibRecommender==0.12.0

  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


## Train

In [ ]:

import tensorflow as tf

from libreco.data import DatasetFeat, DataInfo
from libreco.data import split_by_ratio_chrono,split_by_ratio
from libreco.algorithms import DeepFM,YouTubeRanking
from libreco.evaluation import evaluate
from libreco.prediction import predict_data_with_feats

In [ ]:
rating_df

Index(['user', 'item', 'label', 'daytime', 'weekday', 'isweekend', 'homework',
       'cost', 'weather', 'country', 'city', 'category', 'price', 'rating'],
      dtype='object')

In [ ]:
from numpy.random import noncentral_f

col_sparse= ['daytime', 'weekday', 'isweekend', 'homework',
        'cost','weather', 'country', 'city']
user_col = ['daytime', 'weekday', 'isweekend', 'homework',
       'cost', 'weather', 'country', 'city']
item_col=None
dense_col=None

In [ ]:
# rating_df['label']=1

In [ ]:
train, test = split_by_ratio(rating_df, test_size=0.2)

In [ ]:
train_data, data_info = DatasetFeat.build_trainset(
        train, user_col, item_col, col_sparse, dense_col
    )

In [ ]:
test_data = DatasetFeat.build_testset(test, shuffle=False)

In [ ]:
import torch
from libreco.algorithms import (
    FM,
    WideDeep,
    DeepFM
)


    # # only do negative sampling on eval data
    # eval_data.build_negative_samples(
    #     data_info, item_gen_mode="random", num_neg=1, seed=2222
    # )



# Youtube retrival

In [ ]:

ytb_ranking = YouTubeRanking(task="ranking", data_info=data_info, embed_size=64,
                             n_epochs=20, lr=1e-4, batch_size=256, use_bn=True,
                             hidden_units="128,128,32")
ytb_ranking.fit(train_data, verbose=2, shuffle=True, eval_data=test_data,
                metrics=[ "precision", "recall", "map", "ndcg"])



Instructions for updating:
Colocations handled automatically by placer.


total params: 440,065 | embedding params: 327,457 | network params: 112,608
Training start time: 2023-02-05 11:36:55


train: 100%|██████████| 301/301 [00:03<00:00, 78.23it/s] 


Epoch 1 elapsed: 3.873s
	 train_loss: -426.9019


eval_rec: 100%|██████████| 759/759 [00:02<00:00, 314.57it/s]


...

	 eval precision@10: 0.0001
	 eval recall@10: 0.0001
	 eval map@10: 0.0003
	 eval ndcg@10: 0.0005


train: 100%|██████████| 301/301 [00:02<00:00, 148.69it/s]


Epoch 20 elapsed: 2.049s
	 train_loss: -11192.5225


eval_rec: 100%|██████████| 759/759 [00:01<00:00, 401.49it/s]


	 eval precision@10: 0.0001
	 eval recall@10: 0.0001
	 eval map@10: 0.0013
	 eval ndcg@10: 0.0013


In [ ]:
rating_df.head(1)

,user,item,label,daytime,weekday,isweekend,homework,cost,weather,country,city,category,language
0,0,0,1,morning,sunday,weekend,unknown,free,sunny,United States,0,Productivity,en


In [ ]:
print( "recommendation: ",
        ytb_ranking.recommend_user(
            user=0,
            n_rec=7,
            inner_id=False,
            cold_start="average",
            user_feats=pd.Series({"daytime": "morning", "weekday": "sunday	", "isweekend": "weekend","homework": "unknown", "cost": "free	", "weather": "sunny","country":"United States","city":"0"}),
           
        ))
    

recommendation:  [(1730, 8.649467e-22), (918, 8.630548e-22), (1831, 8.613215e-22), (1172, 8.613117e-22), (1100, 8.554863e-22), (2752, 8.546937e-22), (756, 8.521349e-22)]


In [ ]:
item_df[item_df.item==1831]

,item
1831,1831


## deepFM

In [ ]:
train_data.build_negative_samples(
        data_info, num_neg=1, item_gen_mode="random", seed=2020
    )
test_data.build_negative_samples(
        data_info, num_neg=1, item_gen_mode="random", seed=2222
    )

random neg item sampling elapsed: 0.060s
random neg item sampling elapsed: 0.014s


In [ ]:
deepfm = DeepFM(
        "ranking",
        data_info,
        embed_size=128,
        n_epochs=30,
        lr=1e-4,
        lr_decay=False,
        reg=None,
        batch_size=256,
        num_neg=1,
        use_bn=False,
        dropout_rate=None,
        hidden_units="128,128,32",
        tf_sess_config=None,
    )
    

total params: 1,284,294 | embedding params: 987,195 | network params: 297,099


In [ ]:
deepfm.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=test_data,
        metrics=[
            "loss",
            "balanced_accuracy",
            "roc_auc",
            "pr_auc",
            "precision",
            "recall",
            "map",
            "ndcg",
        ],
    )


Training start time: 2023-02-05 11:41:44


train: 100%|██████████| 602/602 [00:02<00:00, 215.38it/s]


Epoch 1 elapsed: 2.841s
	 train_loss: 0.4281


eval_rec: 100%|██████████| 759/759 [00:01<00:00, 437.92it/s]


	 eval log_loss: 0.3037
	 eval balanced accuracy: 0.8768
	 eval roc_auc: 0.9435
	 eval pr_auc: 0.9467
	 eval precision@10: 0.0241
	 eval recall@10: 0.0596
	 eval map@10: 0.0806
	 eval ndcg@10: 0.1131


...

	 eval log_loss: 0.2147
	 eval balanced accuracy: 0.9204
	 eval roc_auc: 0.9723
	 eval pr_auc: 0.9692
	 eval precision@10: 0.0179
	 eval recall@10: 0.0450
	 eval map@10: 0.0675
	 eval ndcg@10: 0.0902


train: 100%|██████████| 602/602 [00:02<00:00, 228.15it/s]


Epoch 30 elapsed: 2.679s
	 train_loss: 0.1608


eval_rec: 100%|██████████| 759/759 [00:01<00:00, 438.86it/s]


	 eval log_loss: 0.2121
	 eval balanced accuracy: 0.9202
	 eval roc_auc: 0.9722
	 eval pr_auc: 0.9689
	 eval precision@10: 0.0167
	 eval recall@10: 0.0406
	 eval map@10: 0.0653
	 eval ndcg@10: 0.0860


In [ ]:
print( "recommendation: ",
        deepfm.recommend_user(
            user=0,
            n_rec=7,
            inner_id=False,
            cold_start="average",
            user_feats=pd.Series({"daytime": "morning", "weekday": "sunday	", "isweekend": "weekend","homework": "unknown", "cost": "free	", "weather": "sunny","country":"United States","city":"0"}),
           
        ))
    

recommendation:  [(5, 0.9997583), (59, 0.9994455), (38, 0.9992636), (103, 0.9990024), (49, 0.9988986), (11, 0.9986292), (319, 0.9983712)]
